In [1]:
import pandas as pd
% load_ext autoreload
% autoreload 2

In [58]:
from maps.blockgroups import MedianIncome
from maps.elections import USHouseElectionResult
import folium

In [60]:
root_dir = '~/Documents/data'
election_result = USHouseElectionResult(root_dir, '2022-05-24', '007')

In [64]:
mi = MedianIncome(root_dir)
ushouse_7th_median_income = mi.get_median_income(election_result.county_codes)
ushouse_7th_median_income = ushouse_7th_median_income.overlay(election_result.district_map, how='intersection')

In [149]:
ushouse_7th_median_income = ushouse_7th_median_income.assign(
    median_household_income_scaled=ushouse_7th_median_income.median_household_income / 1000)
ushouse_7th_median_income.head()

,GEOID,state,county_fips,county_code,tract,block_group,median_household_income,median_household_income_moe,id,area,district,population,ideal_value,center,geometry,median_household_income_bin,median_household_income_scaled
0,131210116301,13,121,060,011630,1,153487.0,54120.0,258,8.357765e+08,007,765137,765136.0,POINT (-84.05953 33.92267),"POLYGON ((-84.20555 34.06114, -84.20283 34.060...",200,153.487
1,131210114291,13,121,060,011429,1,107692.0,55694.0,258,8.357765e+08,007,765137,765136.0,POINT (-84.05953 33.92267),"POLYGON ((-84.26928 34.02678, -84.26935 34.028...",125,107.692
2,131210114331,13,121,060,011433,1,225152.0,118774.0,258,8.357765e+08,007,765137,765136.0,POINT (-84.05953 33.92267),"POLYGON ((-84.26907 34.01596, -84.26900 34.018...",300,225.152
3,131210116352,13,121,060,011635,2,142011.0,73215.0,258,8.357765e+08,007,765137,765136.0,POINT (-84.05953 33.92267),"POLYGON ((-84.09813 34.05072, -84.09784 34.050...",150,142.011
4,131210116382,13,121,060,011638,2,48393.0,6990.0,258,8.357765e+08,007,765137,765136.0,POINT (-84.05953 33.92267),"POLYGON ((-84.22999 34.04889, -84.22820 34.049...",50,48.393


In [148]:
election_result_map = election_result.district_map
center = election_result_map.center.iloc[0]
m = folium.Map(location=[center.y, center.x])
bbox = election_result_map.bounds.iloc[0]
m.fit_bounds([[bbox.miny, bbox.minx], [bbox.maxy, bbox.maxx]])

folium.Choropleth(
    bins=7,
    geo_data=ushouse_7th_median_income[['GEOID', 'geometry']],
    name="choropleth",
    data=ushouse_7th_median_income[['GEOID', 'median_household_income_scaled']],
    columns=["GEOID", "median_household_income_scaled"],
    key_on="feature.properties.GEOID",
    fill_color="YlGn",
    fill_opacity=0.7,
    line_opacity=0.0,
    line_weight=0,
    legend_name="Median Income ($1000s)",
    highlight=True
).add_to(m)

m

af = election_result.results[['county_code', 'precinct_name', 'party', 'votes']].\
    groupby(['county_code', 'precinct_name', 'party']).\
    sum().reset_index()

af = af.pivot(index=['county_code', 'precinct_name'], columns='party', values='votes')
af.columns.name = None
af = af.reset_index()
af = af.assign(affinity=(af.REP / (af.REP + af.DEM)).apply(lambda x: (max(.4, min(.6, x)) - 0.4) / 0.2),
               pop_scale=(af.REP + af.DEM) / (af.REP + af.DEM).max())

vtd_maps = election_result.vtd_maps

af_marker = af.merge(vtd_maps[['county_code', 'precinct_name', 'center']], on=['county_code', 'precinct_name'],
                     how='inner')
print(af_marker.head())

import plotly.express as px
from plotly.colors import n_colors, find_intermediate_color

for i in af_marker.index:
    center = af_marker.loc[i].center
    x = center.x
    y = center.y
    r = af_marker.loc[i].pop_scale * 20
    c = '#' + ''.join(['%02x' % int(x) for x in find_intermediate_color((0, 0, 255), (255, 0, 0), af_marker.loc[i].affinity)])
    pn = af_marker.loc[i].precinct_name.replace(' ', '&nbsp;')
    rep = af_marker.loc[i].REP
    dem = af_marker.loc[i].DEM
    line2 = f'Rep({rep})&nbsp;vs.&nbsp;Dem({dem})'
    text = f'{pn}<br>{line2}'
    folium.CircleMarker(
        location=[y, x],
        radius=r,
        popup=text,
        color=c,
        fill=True,
        fill_color=c,
        fill_opacity = 0.5,
        opacity = 0.5,
        weight = 0.2,

    ).add_to(m)

m


  county_code precinct_name  DEM  REP  affinity  pop_scale  \
0         060          JC01  430  468  0.605791   0.514024   
1         060         JC01A  166  167  0.507508   0.190612   
2         060          JC02  302  356  0.705167   0.376646   
3         060         JC03A   51   68  0.857143   0.068117   
4         060         JC03B  198  166  0.280220   0.208357   

                       center  
0  POINT (-84.15460 34.04544)  
1  POINT (-84.13240 34.04090)  
2  POINT (-84.16660 34.05487)  
3  POINT (-84.19372 34.06655)  
4  POINT (-84.18597 34.06314)  


(127.5, 0.0, 127.5)

In [88]:
election_result.results.head()

,id,election_date,county_code,contest,choice,party,is_question,precinct_name,votes
0,910737,2022-05-24,060,DEM - US HOUSE DIST 7,CAROLYN BOURDEAUX (I),DEM,0,AP14,8
1,910738,2022-05-24,060,DEM - US HOUSE DIST 7,CAROLYN BOURDEAUX (I),DEM,0,AP14A,47
2,910739,2022-05-24,060,DEM - US HOUSE DIST 7,CAROLYN BOURDEAUX (I),DEM,0,JC01,75
3,910740,2022-05-24,060,DEM - US HOUSE DIST 7,CAROLYN BOURDEAUX (I),DEM,0,JC01A,29
4,910741,2022-05-24,060,DEM - US HOUSE DIST 7,CAROLYN BOURDEAUX (I),DEM,0,JC02,64


In [113]:
af = election_result.results[['county_code', 'precinct_name', 'party', 'votes']].\
    groupby(['county_code', 'precinct_name', 'party']).\
    sum().reset_index()

af = af.pivot(index=['county_code', 'precinct_name'], columns='party', values='votes')
af.columns.name = None
af = af.reset_index()
af = af.assign(affinity=(af.REP / (af.REP + af.DEM)).apply(lambda x: (max(.4, min(.6, x)) - 0.4) / 0.2),
               pop_scale=(af.REP + af.DEM) / (af.REP + af.DEM).max())

vtd_maps = election_result.vtd_maps

In [109]:
af_marker = af.merge(vtd_maps[['county_code', 'precinct_name', 'center']], on=['county_code', 'precinct_name'],
                     how='inner')


In [116]:
import plotly.express as px
from plotly.colors import n_colors, find_intermediate_color

for i in af_marker.index:
    center = af_marker.loc[i].center
    x = center.x
    y = center.y
    r = af_marker.loc[i].pop_scale * 10
    c = find_intermediate_color((255, 0, 0), (0, 0, 255), af_marker.loc[i].affinity)
    pn = af_marker.loc[i].precinct_name
    folium.CircleMarker(
        location=[y, x],
        radius=r,
        popup=pn,
        color=c,
        fill=True,
        fill_color=c
    ).add_to(m)

m




In [56]:
vtd_style_function = lambda x: {
    'color': 'black',
    'weight': 1,
    'min_zoom' = 12,
                 'fillOpacity': 0.0
}

folium.GeoJson(
    vtd_maps[['geometry']],
    name='Precincts Boundaries',
    style_function=vtd_style_function,
).add_to(m)

m